## Learning Realistic Mutations: Bug Creation for Neural Bug Detection
This accompanying notebook is an interactive demo for our contextual mutator. The goal is to inject realistic
bugs into code based on the context.

## Setup preprocessing
As a first step, we setup the preprocessing of the code to be mutated. Here,
we employ a specific syntactic tagger that preserves semantic roles of tokens.

In [27]:
code_example = """

public int additionTest(int a, int b){
    return a + b;
}

"""

from ctxmutants.tokenize import func_tokenize

def tokenize_code(code_snippet):
    try:
        return func_tokenize(code_snippet, lang = "java")
    except Exception:
        return func_tokenize("public class Test{%s}" % code_snippet, lang = "java")


func_to_tokens_types = tokenize_code(code_example)
token_types = next(iter(func_to_tokens_types.values()))

tokens, types = token_types["tokens"], token_types["types"]
print(tokens)

['public', 'int', 'additionTest', '(', 'int', 'a', ',', 'int', 'b', ')', '{', 'return', 'a', '+', 'b', ';', '}']


The `tokenize_code` function has successfully (1) identified the functions in the code snippet, (2) produced a tokenized version of the function and (3) computed syntactic types
of each token. We will investigate the latter.

In [9]:
for i, token in enumerate(tokens):
    token_type = types[i]
    print("%s : %s" % (token, token_type.name))

public : KEYWORDS
int : USE_TYPE
additionTest : DEF_FUNC
( : SYNTAX
int : USE_TYPE
a : DEF_VAR
, : SYNTAX
int : USE_TYPE
b : DEF_VAR
) : SYNTAX
{ : SYNTAX
return : KEYWORDS
a : USE_VAR
+ : BOP
b : USE_VAR
; : SYNTAX
} : SYNTAX


## Setup contextual mutation
The contextual mutation process consists of two steps: (1) we employ
a base mutator to generate mutation candidates and (2) select a mutation
candidate with a masked language model.

In [30]:
import random
from collections import namedtuple

from ctxmutants.mutation import strong_binary_mutation, weak_varmisuse_mutation
from ctxmutants.mutation import weak_binary_mutation
from ctxmutants.mutation import ContextualBatchMutation
from ctxmutants.mutation import mlm_codebert_rerank

MutationCandidate = namedtuple('MutationCandidate', ["tokens", "types", "mask", "location"])

def ctx_mutate(tokens, types, location = None, topK = 5):

    # (1) Select location and base mutator
    base_mutators = {"USE_VAR": weak_varmisuse_mutation, "BOP": strong_binary_mutation}
    if location is None: 
        candidate_locs = [i for i, t in enumerate(types) if t.name in base_mutators]
        assert len(candidate_locs) > 0
        location = random.choice(candidate_locs)
    else:
        assert types[location].name in base_mutators

    base_mutator = base_mutators[types[location].name]

    # (2) Setup mutator with base_mutator and use CodeBert for contextual mutation
    mutator = ContextualBatchMutation(base_mutator, mlm_codebert_rerank, lang = "java")
    mutant_dist = mutator(
        [MutationCandidate(tokens, types, None, location)]
    )
    mutant_dist = mutant_dist[0]

    # (3) We report the topK mutants
    topK_mutants = sorted(mutant_dist.items(), key = lambda x: x[1], reverse = True)
    topK_mutants = topK_mutants[:topK]

    output = []

    for mutant, prob in topK_mutants:
        mutant_tokens = list(tokens)
        mutant_tokens[location] = mutant
        mutant_code = " ".join(mutant_tokens)

        output.append({
            "code": mutant_code,
            "location": location,
            "before" : tokens[location],
            "after": mutant,
            "score" : prob
        })


    return output


ctx_mutate(tokens, types, location = 13)


[{'code': 'public int additionTest ( int a , int b ) { return a > b ; }',
  'location': 13,
  'before': '+',
  'after': '>',
  'score': 0.3605694422459369},
 {'code': 'public int additionTest ( int a , int b ) { return a < b ; }',
  'location': 13,
  'before': '+',
  'after': '<',
  'score': 0.1880156641423118},
 {'code': 'public int additionTest ( int a , int b ) { return a >= b ; }',
  'location': 13,
  'before': '+',
  'after': '>=',
  'score': 0.1617198824838041},
 {'code': 'public int additionTest ( int a , int b ) { return a - b ; }',
  'location': 13,
  'before': '+',
  'after': '-',
  'score': 0.1067578773077604},
 {'code': 'public int additionTest ( int a , int b ) { return a == b ; }',
  'location': 13,
  'before': '+',
  'after': '==',
  'score': 0.07315896729156009}]

## Playground
Here we have a small playground to use our contextual mutator

In [41]:
source_code = """

public void test(Object input) {
    if(input == null) {
        return;
    }
    handle(input);
}

"""

tokens_types  = next(iter(tokenize_code(source_code).values()))
tokens, types = tokens_types["tokens"], tokens_types["types"]

print("Tokens:")
print(tokens)
print()
print("Available mutation locations:")

for i, token in enumerate(tokens):
    type = types[i]
    if type.name in ["USE_VAR", "BOP"]:
        token_ctx = " ".join(tokens[i-3:i+3])
        print("Loc %d | Token: %s  | Type: %s | Ctx: %s" % (i, token, type.name, token_ctx))

Tokens:
['public', 'void', 'test', '(', 'Object', 'input', ')', '{', 'if', '(', 'input', '==', 'null', ')', '{', 'return', ';', '}', 'handle', '(', 'input', ')', ';', '}']

Available mutation locations:
Loc 10 | Token: input  | Type: USE_VAR | Ctx: { if ( input == null
Loc 11 | Token: ==  | Type: BOP | Ctx: if ( input == null )
Loc 20 | Token: input  | Type: USE_VAR | Ctx: } handle ( input ) ;


Choose a mutant location and the contextual mutator will produce some candidates.

In [42]:
mutant_location = 11 # Change here (Set to None if random)

mutations = ctx_mutate(tokens, types, location = mutant_location)

for mutation in mutations:
    print(mutation)
    print("----")


{'code': 'public void test ( Object input ) { if ( input != null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '!=', 'score': 0.9624839110581414}
----
{'code': 'public void test ( Object input ) { if ( input <= null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '<=', 'score': 0.027384249670528885}
----
{'code': 'public void test ( Object input ) { if ( input < null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '<', 'score': 0.008271744622112507}
----
{'code': 'public void test ( Object input ) { if ( input >= null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '>=', 'score': 0.000676133948626892}
----
{'code': 'public void test ( Object input ) { if ( input > null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '>', 'score': 0.0005877486022105648}
----


### Comparison between mutation types
For your example in playgrounds, we will now compute representative
mutants for each mutation type.

In [52]:
# For sampling a mutation
def sample_mutation(mutation_dist):
    random_selection = random.random()
    cumsum = 0.0

    for key, prob in sorted(mutation_dist.items(), key = lambda x: x[1], reverse=True):
        if cumsum + prob >= random_selection: return key
        cumsum += prob
    
    return key # Because of numerical instabilities sum(probs) smaller 1.0


assert types[mutant_location].name == "BOP", "Only all three mutation types are supported for BOP"

# Loose mutant
loose_mutation_dist = weak_binary_mutation(tokens, mutant_location)
loose_mutant = sample_mutation(loose_mutation_dist)

# Strict mutant
strong_mutation_dist = strong_binary_mutation(tokens, mutant_location)
strong_mutant = sample_mutation(strong_mutation_dist)

# Contextual mutant

ctx_mutation_dist = {m["after"] : m["score"] for m in mutations}
ctx_mutant = sample_mutation(ctx_mutation_dist)

for mutant_type, mutant in [("Loose mutation", loose_mutant) , ("Strict mutation", strong_mutant), ("Contextual mutation", ctx_mutant)]:
    print(mutant_type)
    print("----------------------------------------------------------------")

    mutate_tokens = list(tokens)
    mutate_tokens[mutant_location] = mutant
    mutate_code = " ".join(mutate_tokens)

    print(mutate_code)
    print()


Loose mutation
----------------------------------------------------------------
public void test ( Object input ) { if ( input + null ) { return ; } handle ( input ) ; }

Strict mutation
----------------------------------------------------------------
public void test ( Object input ) { if ( input >= null ) { return ; } handle ( input ) ; }

Contextual mutation
----------------------------------------------------------------
public void test ( Object input ) { if ( input != null ) { return ; } handle ( input ) ; }

